In [5]:
import pandas as pd

In [6]:
df_price = pd.read_csv("../../data/raw/movie_price_spreadsheets.csv", engine= "python", encoding='utf-8')
df_download = pd.read_csv("../../data/cleaned/movie_download_cleaned.csv", engine= "python", encoding='utf-8')
df_customer = pd.read_csv("../../data/cleaned/movie_customer_cleaned.csv", engine= "python", encoding='utf-8')

In [7]:
df_elasticity = pd.DataFrame([2014, 2015, 2016, 2017, 2018], columns=['year'])
df_elasticity

,year
0,2014
1,2015
2,2016
3,2017
4,2018


In [8]:
df_price.describe()

,contract_year,release_year,runtime,imdb_score,votes,reviews_users,reviews_critics,budget,income_usa,income_ww,theater_opening,theater_total,price_year,basic_price,dc_rate,subscribe_price,contract_price,studio_score,price_class
count,394.000000,394.000000,394.000000,394.000000,3.940000e+02,394.000000,394.000000,3.940000e+02,3.940000e+02,3.940000e+02,394.000000,394.000000,394.000000,394.000000,394.000000,394.000000,394.000000,394.000000,394.000000
mean,2014.756345,2012.497462,126.345178,7.183756,5.081580e+05,1254.784264,463.908629,1.484695e+08,2.995764e+08,7.708920e+08,3775.157360,3985.236041,2016.378173,1.020305,0.075508,0.943528,1220.604061,8.304569,5.017766
std,1.070789,3.551331,22.164608,0.842322,3.495355e+05,1007.944782,160.927581,6.913259e+07,1.141374e+08,3.705083e+08,870.286432,298.981260,1.342828,0.068750,0.038289,0.078432,91.745558,1.979187,4.463843
min,2014.000000,2005.000000,87.000000,4.100000,3.990000e+04,148.000000,151.000000,4.500000e+06,1.102127e+08,1.407958e+08,1.000000,3142.000000,2014.000000,0.900000,0.000000,0.810000,955.000000,6.000000,1.000000
25%,2014.000000,2009.000000,105.250000,6.700000,2.683822e+05,511.000000,345.000000,8.000000e+07,2.299287e+08,5.529826e+08,3702.000000,3796.000000,2015.000000,1.000000,0.050000,0.900000,1155.000000,6.000000,1.000000
50%,2014.000000,2013.000000,127.000000,7.300000,4.717730e+05,1001.000000,429.500000,1.500000e+08,2.925762e+08,7.450131e+08,3997.000000,4022.500000,2017.000000,1.000000,0.100000,0.900000,1299.000000,10.000000,1.000000
75%,2015.000000,2015.000000,144.000000,7.800000,6.465240e+05,1549.000000,579.000000,2.000000e+08,3.479118e+08,9.385804e+08,4207.000000,4207.000000,2018.000000,1.000000,0.100000,1.000000,1299.000000,10.000000,10.000000
max,2018.000000,2018.000000,169.000000,9.000000,2.134569e+06,6344.000000,837.000000,3.210000e+08,7.605076e+08,2.789968e+09,4529.000000,4535.000000,2018.000000,1.200000,0.100000,1.200000,1299.000000,10.000000,10.000000


In [9]:
# 연당 평균 다운로드 가격
df_pr_mean_year = pd.DataFrame([df_price['subscribe_price'].groupby(df_price['price_year']).mean()])
df_pr_mean_year

price_year,2014,2015,2016,2017,2018
subscribe_price,0.927222,0.958507,0.95128,0.941383,0.936887


In [10]:
# 누적 등록 회원 수
num_register = pd.DataFrame([df_customer['register_year'].value_counts()]).sort_index(axis=1)
num_register_cum = num_register.cumsum(axis = 1)
num_register_cum

,2014,2015,2016,2017,2018
register_year,1017,1745,2407,3268,3277


In [11]:
# 연당 탈퇴 회원 수
num_drop = pd.DataFrame([df_customer['drop_year'].value_counts()[1:]])
num_drop = num_drop.reindex(sorted(num_drop.columns), axis=1)
num_drop

,2015,2016,2017,2018
drop_year,130,143,189,197


In [12]:
# 연간 순 활동 회원 수(active)
num_custom_year = []
num_custom_year.append(int(num_register_cum[2014]))


num_custom_year.append(int(num_register_cum[2015]) - int(num_drop['2015']))
num_custom_year.append(int(num_register_cum[2016]) - int(num_drop['2016']))
num_custom_year.append(int(num_register_cum[2017]) - int(num_drop['2017']))
num_custom_year.append(int(num_register_cum[2018]) - int(num_drop['2018']))

num_custom_year = pd.DataFrame([num_custom_year], columns=num_register_cum.columns, index=['active_user']).sort_index(axis=1)
num_custom_year

,2014,2015,2016,2017,2018
active_user,1017,1615,2264,3079,3080


In [13]:
df_main = pd.DataFrame([df_download['down_year'].value_counts()], index=['down_cnt'])
df_main = df_main.reindex(sorted(df_main.columns), axis=1).transpose()

In [14]:
df_main['active_user'] = num_custom_year.transpose()

In [15]:
df_main['per_capita_down'] = df_main['down_cnt']/df_main['active_user']
df_main['price_mean'] = df_pr_mean_year.transpose()

In [16]:
d_price_mean = [0]
for i in range(2015,2019):
    d_price_mean.append(df_main['price_mean'][i] - df_main['price_mean'][i-1])
    
d_per_capita_down = [0]
for i in range(2015,2019):
    d_per_capita_down.append(df_main['per_capita_down'][i] - df_main['per_capita_down'][i-1])

df_main['d_price_mean'] = d_price_mean
df_main['d_per_capita_down'] = d_per_capita_down

In [22]:
elasticity = [0]
for i in range(2015,2019):
    elasticity.append((df_main['d_per_capita_down'][i] / df_main['per_capita_down'][i])/(df_main['d_price_mean'][i] / df_main['price_mean'][i]))
df_main['elasticity_all'] = elasticity
df_elasticity['all'] = elasticity

In [23]:
df_main

,down_cnt,active_user,per_capita_down,price_mean,d_price_mean,d_per_capita_down,elasticity_all
2014,5031,1017,4.946903,0.927222,0.000000,0.000000,0.000000
2015,11981,1615,7.418576,0.958507,0.031285,2.471673,10.207667
2016,15063,2264,6.653269,0.951280,-0.007227,-0.765307,15.140940
2017,21126,3079,6.861319,0.941383,-0.009898,0.208050,-2.884036
2018,19884,3080,6.455844,0.936887,-0.004496,-0.405474,13.087397


# 고객 특성별 탄력성 구하기

## 여성 고객

In [24]:
df_price = pd.read_csv("../../data/raw/movie_price_spreadsheets.csv", engine= "python", encoding='utf-8')
df_download = pd.read_csv("../../data/cleaned/movie_download_cleaned.csv", engine= "python", encoding='utf-8')
df_customer = pd.read_csv("../../data/cleaned/movie_customer_cleaned.csv", engine= "python", encoding='utf-8')



df_customer = df_customer[df_customer['gender'] == 'F']
df_download = df_download[df_download['customer_id'].isin(df_customer['customer_id'])]
df_price = df_price[df_price['item_id'].isin(df_download['item_id'])]


# 연당 평균 다운로드 가격
df_pr_mean_year = pd.DataFrame([df_price['subscribe_price'].groupby(df_price['price_year']).mean()])
df_pr_mean_year
# 누적 등록 회원 수
num_register = pd.DataFrame([df_customer['register_year'].value_counts()]).sort_index(axis=1)
num_register_cum = num_register.cumsum(axis = 1)
num_register_cum
# 연당 탈퇴 회원 수
num_drop = pd.DataFrame([df_customer['drop_year'].value_counts()[1:]])
num_drop = num_drop.reindex(sorted(num_drop.columns), axis=1)
num_drop
# 연간 순 활동 회원 수(active)
num_custom_year = []
num_custom_year.append(int(num_register_cum[2014]))


num_custom_year.append(int(num_register_cum[2015]) - int(num_drop['2015']))
num_custom_year.append(int(num_register_cum[2016]) - int(num_drop['2016']))
num_custom_year.append(int(num_register_cum[2017]) - int(num_drop['2017']))
num_custom_year.append(int(num_register_cum[2018]) - int(num_drop['2018']))

num_custom_year = pd.DataFrame([num_custom_year], columns=num_register_cum.columns, index=['active_user']).sort_index(axis=1)
num_custom_year

df_main = pd.DataFrame([df_download['down_year'].value_counts()], index=['down_cnt'])
df_main = df_main.reindex(sorted(df_main.columns), axis=1).transpose()

df_main['active_user'] = num_custom_year.transpose()

df_main['per_capita_down'] = df_main['down_cnt']/df_main['active_user']
df_main['price_mean'] = df_pr_mean_year.transpose()

d_price_mean = [0]
for i in range(2015,2019):
    d_price_mean.append(df_main['price_mean'][i] - df_main['price_mean'][i-1])
    
d_per_capita_down = [0]
for i in range(2015,2019):
    d_per_capita_down.append(df_main['per_capita_down'][i] - df_main['per_capita_down'][i-1])

df_main['d_price_mean'] = d_price_mean
df_main['d_per_capita_down'] = d_per_capita_down

elasticity = [0]
for i in range(2015,2019):
    elasticity.append((df_main['d_per_capita_down'][i] / df_main['per_capita_down'][i])/(df_main['d_price_mean'][i] / df_main['price_mean'][i]))
df_main['elasticity_female'] = elasticity
df_elasticity['female'] = elasticity

df_main

,down_cnt,active_user,per_capita_down,price_mean,d_price_mean,d_per_capita_down,elasticity_female
2014,2174,496,4.383065,0.927222,0.000000,0.000000,0.000000
2015,5726,800,7.157500,0.958507,0.031285,2.774435,11.875975
2016,7951,1124,7.073843,0.951280,-0.007227,-0.083657,1.556671
2017,11190,1554,7.200772,0.941383,-0.009898,0.126929,-1.676569
2018,10353,1552,6.670747,0.936887,-0.004496,-0.530025,16.556346


# 남성 고객

In [32]:
df_price = pd.read_csv("../../data/raw/movie_price_spreadsheets.csv", engine= "python", encoding='utf-8')
df_download = pd.read_csv("../../data/cleaned/movie_download_cleaned.csv", engine= "python", encoding='utf-8')
df_customer = pd.read_csv("../../data/cleaned/movie_customer_cleaned.csv", engine= "python", encoding='utf-8')



df_customer = df_customer[df_customer['gender'] == 'M']
df_download = df_download[df_download['customer_id'].isin(df_customer['customer_id'])]
df_price = df_price[df_price['item_id'].isin(df_download['item_id'])]


# 연당 평균 다운로드 가격
df_pr_mean_year = pd.DataFrame([df_price['subscribe_price'].groupby(df_price['price_year']).mean()])
df_pr_mean_year
# 누적 등록 회원 수
num_register = pd.DataFrame([df_customer['register_year'].value_counts()]).sort_index(axis=1)
num_register_cum = num_register.cumsum(axis = 1)
num_register_cum
# 연당 탈퇴 회원 수
num_drop = pd.DataFrame([df_customer['drop_year'].value_counts()[1:]])
num_drop = num_drop.reindex(sorted(num_drop.columns), axis=1)
num_drop
# 연간 순 활동 회원 수(active)
num_custom_year = []
num_custom_year.append(int(num_register_cum[2014]))


num_custom_year.append(int(num_register_cum[2015]) - int(num_drop['2015']))
num_custom_year.append(int(num_register_cum[2016]) - int(num_drop['2016']))
num_custom_year.append(int(num_register_cum[2017]) - int(num_drop['2017']))
num_custom_year.append(int(num_register_cum[2018]) - int(num_drop['2018']))

num_custom_year = pd.DataFrame([num_custom_year], columns=num_register_cum.columns, index=['active_user']).sort_index(axis=1)
num_custom_year

df_main = pd.DataFrame([df_download['down_year'].value_counts()], index=['down_cnt'])
df_main = df_main.reindex(sorted(df_main.columns), axis=1).transpose()

df_main['active_user'] = num_custom_year.transpose()

df_main['per_capita_down'] = df_main['down_cnt']/df_main['active_user']
df_main['price_mean'] = df_pr_mean_year.transpose()

d_price_mean = [0]
for i in range(2015,2019):
    d_price_mean.append(df_main['price_mean'][i] - df_main['price_mean'][i-1])
    
d_per_capita_down = [0]
for i in range(2015,2019):
    d_per_capita_down.append(df_main['per_capita_down'][i] - df_main['per_capita_down'][i-1])

df_main['d_price_mean'] = d_price_mean
df_main['d_per_capita_down'] = d_per_capita_down

elasticity = [0]
for i in range(2015,2019):
    elasticity.append((df_main['d_per_capita_down'][i] / df_main['per_capita_down'][i])/(df_main['d_price_mean'][i] / df_main['price_mean'][i]))
df_main['elasticity_male'] = elasticity
df_elasticity['male'] = elasticity

df_main

,down_cnt,active_user,per_capita_down,price_mean,d_price_mean,d_per_capita_down,elasticity_male
2014,2857,521,5.483685,0.927222,0.000000,0.000000,0.000000
2015,6255,815,7.674847,0.958507,0.031285,2.191161,8.747031
2016,7112,1140,6.238596,0.951280,-0.007227,-1.436250,30.303670
2017,9936,1525,6.515410,0.941383,-0.009898,0.276813,-4.040971
2018,9531,1528,6.237565,0.936887,-0.004496,-0.277844,9.281739


## 12세 미만 어린이 O

In [33]:
df_price = pd.read_csv("../../data/raw/movie_price_spreadsheets.csv", engine= "python", encoding='utf-8')
df_download = pd.read_csv("../../data/cleaned/movie_download_cleaned.csv", engine= "python", encoding='utf-8')
df_customer = pd.read_csv("../../data/cleaned/movie_customer_cleaned.csv", engine= "python", encoding='utf-8')



df_customer = df_customer[df_customer['kids_under12'] == 'Y']
df_download = df_download[df_download['customer_id'].isin(df_customer['customer_id'])]
df_price = df_price[df_price['item_id'].isin(df_download['item_id'])]


# 연당 평균 다운로드 가격
df_pr_mean_year = pd.DataFrame([df_price['subscribe_price'].groupby(df_price['price_year']).mean()])
df_pr_mean_year
# 누적 등록 회원 수
num_register = pd.DataFrame([df_customer['register_year'].value_counts()]).sort_index(axis=1)
num_register_cum = num_register.cumsum(axis = 1)
num_register_cum
# 연당 탈퇴 회원 수
num_drop = pd.DataFrame([df_customer['drop_year'].value_counts()[1:]])
num_drop = num_drop.reindex(sorted(num_drop.columns), axis=1)
num_drop
# 연간 순 활동 회원 수(active)
num_custom_year = []
num_custom_year.append(int(num_register_cum[2014]))


num_custom_year.append(int(num_register_cum[2015]) - int(num_drop['2015']))
num_custom_year.append(int(num_register_cum[2016]) - int(num_drop['2016']))
num_custom_year.append(int(num_register_cum[2017]) - int(num_drop['2017']))
num_custom_year.append(int(num_register_cum[2018]) - int(num_drop['2018']))

num_custom_year = pd.DataFrame([num_custom_year], columns=num_register_cum.columns, index=['active_user']).sort_index(axis=1)
num_custom_year

df_main = pd.DataFrame([df_download['down_year'].value_counts()], index=['down_cnt'])
df_main = df_main.reindex(sorted(df_main.columns), axis=1).transpose()

df_main['active_user'] = num_custom_year.transpose()

df_main['per_capita_down'] = df_main['down_cnt']/df_main['active_user']
df_main['price_mean'] = df_pr_mean_year.transpose()

d_price_mean = [0]
for i in range(2015,2019):
    d_price_mean.append(df_main['price_mean'][i] - df_main['price_mean'][i-1])
    
d_per_capita_down = [0]
for i in range(2015,2019):
    d_per_capita_down.append(df_main['per_capita_down'][i] - df_main['per_capita_down'][i-1])

df_main['d_price_mean'] = d_price_mean
df_main['d_per_capita_down'] = d_per_capita_down

elasticity = [0]
for i in range(2015,2019):
    elasticity.append((df_main['d_per_capita_down'][i] / df_main['per_capita_down'][i])/(df_main['d_price_mean'][i] / df_main['price_mean'][i]))
df_main['elasticity_kids_under12'] = elasticity
df_elasticity['kids_under12'] = elasticity

df_main

,down_cnt,active_user,per_capita_down,price_mean,d_price_mean,d_per_capita_down,elasticity_kids_under12
2014,1856,369,5.029810,0.927222,0.000000,0.000000,0.000000
2015,4419,592,7.464527,0.958507,0.031285,2.434717,9.993144
2016,5363,805,6.662112,0.951280,-0.007227,-0.802415,15.854016
2017,7730,1086,7.117864,0.941383,-0.009898,0.455752,-6.090028
2018,6815,1086,6.275322,0.936887,-0.004496,-0.842541,27.976801


## 12세 미만 어린이 X

In [34]:
df_price = pd.read_csv("../../data/raw/movie_price_spreadsheets.csv", engine= "python", encoding='utf-8')
df_download = pd.read_csv("../../data/cleaned/movie_download_cleaned.csv", engine= "python", encoding='utf-8')
df_customer = pd.read_csv("../../data/cleaned/movie_customer_cleaned.csv", engine= "python", encoding='utf-8')



df_customer = df_customer[df_customer['kids_under12'] == 'N']
df_download = df_download[df_download['customer_id'].isin(df_customer['customer_id'])]
df_price = df_price[df_price['item_id'].isin(df_download['item_id'])]


# 연당 평균 다운로드 가격
df_pr_mean_year = pd.DataFrame([df_price['subscribe_price'].groupby(df_price['price_year']).mean()])
df_pr_mean_year
# 누적 등록 회원 수
num_register = pd.DataFrame([df_customer['register_year'].value_counts()]).sort_index(axis=1)
num_register_cum = num_register.cumsum(axis = 1)
num_register_cum
# 연당 탈퇴 회원 수
num_drop = pd.DataFrame([df_customer['drop_year'].value_counts()[1:]])
num_drop = num_drop.reindex(sorted(num_drop.columns), axis=1)
num_drop
# 연간 순 활동 회원 수(active)
num_custom_year = []
num_custom_year.append(int(num_register_cum[2014]))


num_custom_year.append(int(num_register_cum[2015]) - int(num_drop['2015']))
num_custom_year.append(int(num_register_cum[2016]) - int(num_drop['2016']))
num_custom_year.append(int(num_register_cum[2017]) - int(num_drop['2017']))
num_custom_year.append(int(num_register_cum[2018]) - int(num_drop['2018']))

num_custom_year = pd.DataFrame([num_custom_year], columns=num_register_cum.columns, index=['active_user']).sort_index(axis=1)
num_custom_year

df_main = pd.DataFrame([df_download['down_year'].value_counts()], index=['down_cnt'])
df_main = df_main.reindex(sorted(df_main.columns), axis=1).transpose()

df_main['active_user'] = num_custom_year.transpose()

df_main['per_capita_down'] = df_main['down_cnt']/df_main['active_user']
df_main['price_mean'] = df_pr_mean_year.transpose()

d_price_mean = [0]
for i in range(2015,2019):
    d_price_mean.append(df_main['price_mean'][i] - df_main['price_mean'][i-1])
    
d_per_capita_down = [0]
for i in range(2015,2019):
    d_per_capita_down.append(df_main['per_capita_down'][i] - df_main['per_capita_down'][i-1])

df_main['d_price_mean'] = d_price_mean
df_main['d_per_capita_down'] = d_per_capita_down

elasticity = [0]
for i in range(2015,2019):
    elasticity.append((df_main['d_per_capita_down'][i] / df_main['per_capita_down'][i])/(df_main['d_price_mean'][i] / df_main['price_mean'][i]))
df_main['elasticity_kids_under12_no'] = elasticity
df_elasticity['kids_under12_no'] = elasticity

df_main

,down_cnt,active_user,per_capita_down,price_mean,d_price_mean,d_per_capita_down,elasticity_kids_under12_no
2014,3175,648,4.899691,0.927222,0.000000,0.000000,0.000000
2015,7562,1023,7.391984,0.958507,0.031285,2.492293,10.329851
2016,9700,1459,6.648389,0.951280,-0.007227,-0.743595,14.722179
2017,13396,1993,6.721525,0.941383,-0.009898,0.073136,-1.034913
2018,13069,1994,6.554162,0.936887,-0.004496,-0.167363,5.320895


In [55]:
df_price = pd.read_csv("../../data/raw/movie_price_spreadsheets.csv", engine= "python", encoding='utf-8')
df_download = pd.read_csv("../../data/cleaned/movie_download_cleaned.csv", engine= "python", encoding='utf-8')
df_customer = pd.read_csv("../../data/cleaned/movie_customer_cleaned.csv", engine= "python", encoding='utf-8')



df_customer = df_customer[df_customer['drop_flag'] == 'N']
df_download = df_download[df_download['customer_id'].isin(df_customer['customer_id'])]
df_price = df_price[df_price['item_id'].isin(df_download['item_id'])]


# 연당 평균 다운로드 가격
df_pr_mean_year = pd.DataFrame([df_price['subscribe_price'].groupby(df_price['price_year']).mean()])
df_pr_mean_year
# 누적 등록 회원 수
num_register = pd.DataFrame([df_customer['register_year'].value_counts()]).sort_index(axis=1)
num_register_cum = num_register.cumsum(axis = 1)
num_register_cum
# 연당 탈퇴 회원 수
num_drop = pd.DataFrame([df_customer['drop_year'].value_counts()[1:]])
num_drop = num_drop.reindex(sorted(num_drop.columns), axis=1)
num_drop
# 연간 순 활동 회원 수(active)
num_custom_year = []
num_custom_year.append(int(num_register_cum[2014]))


num_custom_year.append(int(num_register_cum[2015]) - int(num_drop['2015']))
num_custom_year.append(int(num_register_cum[2016]) - int(num_drop['2016']))
num_custom_year.append(int(num_register_cum[2017]) - int(num_drop['2017']))
num_custom_year.append(int(num_register_cum[2018]) - int(num_drop['2018']))

num_custom_year = pd.DataFrame([num_custom_year], columns=num_register_cum.columns, index=['active_user']).sort_index(axis=1)
num_custom_year

df_main = pd.DataFrame([df_download['down_year'].value_counts()], index=['down_cnt'])
df_main = df_main.reindex(sorted(df_main.columns), axis=1).transpose()

df_main['active_user'] = num_custom_year.transpose()

df_main['per_capita_down'] = df_main['down_cnt']/df_main['active_user']
df_main['price_mean'] = df_pr_mean_year.transpose()

d_price_mean = [0]
for i in range(2015,2019):
    d_price_mean.append(df_main['price_mean'][i] - df_main['price_mean'][i-1])
    
d_per_capita_down = [0]
for i in range(2015,2019):
    d_per_capita_down.append(df_main['per_capita_down'][i] - df_main['per_capita_down'][i-1])

df_main['d_price_mean'] = d_price_mean
df_main['d_per_capita_down'] = d_per_capita_down

elasticity = [0]
for i in range(2015,2019):
    elasticity.append((df_main['d_per_capita_down'][i] / df_main['per_capita_down'][i])/(df_main['d_price_mean'][i] / df_main['price_mean'][i]))
df_main['elasticity_active'] = elasticity
df_elasticity['active'] = elasticity

df_main

KeyError: '2016'

# 그냥 매번 코드 새로 안만들고 덮어씀

In [49]:
# df_elasticity = df_elasticity.set_index('year')
df_elasticity

,all,female,male,kids_under12,kids_under12_no,30s,40s,50s,60s+
year,,,,,,,,,
2014,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2015,10.207667,11.875975,8.747031,9.993144,10.329851,10.865979,9.263602,7.788482,7.669608
2016,15.140940,1.556671,30.303670,15.854016,14.722179,15.155869,18.464147,14.047064,17.827832
2017,-2.884036,-1.676569,-4.040971,-6.090028,-1.034913,-10.248524,-2.420584,-0.958338,0.836872
2018,13.087397,16.556346,9.281739,27.976801,5.320895,25.977448,16.614541,14.633478,9.857688


In [30]:
df_customer

,customer_id,gender,state,age,age_group,married,kids_under12,register_year,drop_flag,drop_year,last_down_date,down_flag
1,C5002,M,CA,23,20s,S,N,2014,N,.,10-25-2016,1
7,C5008,M,GA,39,30s,M,Y,2014,N,.,12-26-2015,1
8,C5009,M,VA,30,30s,M,Y,2014,Y,2015,11-22-2014,1
9,C5010,M,TN,32,30s,M,N,2014,N,.,12-27-2014,1
13,C5014,M,TX,44,40s,M,Y,2014,N,.,12-23-2016,1
...,...,...,...,...,...,...,...,...,...,...,...,...
3261,C8262,M,CA,56,50s,M,N,2017,N,.,12-11-2018,1
3264,C8265,M,PA,51,50s,M,Y,2017,N,.,12-13-2017,1
3267,C8268,M,FL,38,30s,M,Y,2018,N,.,12-27-2018,1
3268,C8269,M,MI,35,30s,S,N,2018,N,.,11-26-2018,1
